# Element Parameter Detection

## Setup

In [ ]:
import numpy as np
import math
import random
import os
import os.path
import torch
import sys
import copy
import pickle
import importlib
import torch.nn as nn
import time
import functorch

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
from chamferdist import ChamferDistance
from pathlib import Path

import ifcopenshell
import open3d as o3d

from src.elements import *
from src.ifc import *
from src. preparation import *
from src.dataset import *
from src.pointnet import *
from src.visualisation import *
from src.geometry import sq_distance
from src.icp import icp_finetuning
from src.chamfer import *
from src.utils import *
from src.plots import plot_error_graph, plot_parameter_errors
from src.pca import testset_PCA
#from src.finetune import chamfer_fine_tune
from tqdm.notebook import tqdm


In [ ]:
random.seed = 42

In [ ]:
#path = Path("ModelNet10")
#path = Path('/content/drive/MyDrive/ElementNet/')
path = Path('output/')
#savepath = '/content/drive/MyDrive/ElementNet/'
savepath = 'models/'
cuda = torch.device('cuda')

## Model

## Test

Analyze results statistically

POINTNET++

In [ ]:
train_transforms = transforms.Compose([
                    Normalize(),
#                    RandomNoise(),
                    ToTensor()
                    ])

In [ ]:
# load data and model
BASE_DIR = os.path.dirname(os.path.abspath('industrial-facility-relationships/'))
BASE_DIR = os.path.join(BASE_DIR, 'pointnet2')
ROOT_DIR = BASE_DIR
sys.path.append(os.path.join(ROOT_DIR, 'models'))
inference = True

if inference:
#     path = Path('output/bp_data/')
#     ext = ".ply"
    path = Path('output/west_ref/')
    #path = Path('/mnt/c/data/3D_CAD/east_clouds/')
    ext = ".pcd"
else:
    path = Path('output/')
    ext = ".pcd"

cat= 'elbow'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_normals = False
cat_targets = {"elbow":14, "bend":14, "tee":19, "pipe":11, "flange":13}

if inference:
    test_ds = PointCloudData(path, valid=True, folder='test', category=cat, transform=train_transforms, inference=True)
    targets = cat_targets[cat]
else:
    test_ds = PointCloudData(path, valid=True, folder='test', category=cat, transform=train_transforms)
    targets = test_ds.targets

testDataLoader = torch.utils.data.DataLoader(dataset=test_ds, batch_size=32)
test_criterion = nn.MSELoss()

model_name = "pointnet2_cls_ssg"
model_path = Path("pointnet2/log/classification/pointnet2_cls_ssg/")
model = importlib.import_module(model_name)


predictor = model.get_model(targets, normal_channel=use_normals)
if device != "cpu":
    predictor = predictor.cuda()

#checkpoint = torch.load(model_path/'checkpoints/best_model.pth')
checkpoint = torch.load(model_path/'checkpoints/models/best_model_e_chamfer_0005.pth')
#checkpoint = torch.load(model_path/'checkpoints/models/best_model_t_chamfer_00005_bp.pth')
predictor.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
blueprint = 'data/sample.ifc'
temp_dir = "output/temp/"
target_dir = "output/tee/test/"

ifcConvert_executable = "scripts/./IfcConvert"
cloudCompare_executable = "cloudcompare.CloudCompare"
sample_size = 2048
threshold = 2

In [ ]:
def model_inference(model, loader, device, calculate_score=False):
    predictor = model.eval()
    predictions_list, pcd_list, id_list = [], [], []
    with torch.no_grad():
        for j, data  in tqdm(enumerate(loader), total=len(loader)):
            points, ids = data['pointcloud'].to(device).float(), data['id'].to(device)
            points = points.transpose(2, 1)
            preds, _ = predictor(points)
            preds, points, ids = preds.to(torch.device('cpu')), points.to(torch.device('cpu')), data['id'].to(torch.device('cpu'))
            for i, pr in enumerate(preds):
                predictions_list.append(pr.numpy())
                pcd_list.append(points[i].numpy())
                id_list.append(ids[i].numpy())

        return (predictions_list, pcd_list, id_list)

In [ ]:
if inference:
    predictions_list, cloud_list, id_list = model_inference(predictor.eval(), testDataLoader, device)

In [ ]:
def test(model, loader, device, criterion):
    losses = []
    predictor = model.eval()
    cloud_list = []
    label_list = []
    output_list = []
    predictions_list = []
    inputs_list = []
    id_list = []
    parameter_id = 0
    tot = 0
    count = 0
    
    for j, data  in tqdm(enumerate(loader), total=len(loader)):
        inputs, labels, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)
        points, target, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)
        points = points.transpose(2, 1)
        outputs, _ = predictor(points)
        outputs = outputs.to(torch.device('cpu'))
        inputs = points.to(torch.device('cpu'))
        labels = target.to(torch.device('cpu'))
        ids = ids.to(torch.device('cpu'))
        #print(data['pointcloud'].size(), labels.size(), outputs.size())

        for i in range(outputs.size(0)):
            label_list.append(labels[i][parameter_id].item())
            id_list.append(ids[i].item())
            output_list.append(outputs[i][parameter_id].item())
            predictions_list.append(outputs[i].numpy())
            inputs_list.append(labels[i].numpy())
            cloud_list.append(inputs[i].numpy())
            ratio = ((labels[i][parameter_id]-outputs[i][parameter_id])/labels[i][parameter_id]).item()
            #print('r', i+count, ids[i].item(), labels[i][parameter_id].item(), outputs[i][parameter_id].item(), ratio)
            tot += np.absolute(ratio)
            #print('l', labels[i][1].item(), outputs[i][1].item(), ((labels[i][1]-outputs[i][1])/labels[i][1]).item())
        
        count += outputs.size(0)
    print(tot/count)

    return predictions_list, inputs_list, label_list, output_list, id_list, cloud_list

In [ ]:
if not inference:
    with torch.no_grad():
        predictions_list, inputs_list, label_list, output_list, id_list, cloud_list = test(predictor.eval(), testDataLoader, device, test_criterion)

    print(len(predictions_list), len(inputs_list))

In [ ]:
if not inference:
    label_list, output_list, id_list = np.array(label_list), np.array(output_list), np.array(id_list)

#### Visually analyse predictions and Fine tune with ICP, calculate chamfer distances

In [ ]:
# scaling up and down is required for icp calculations
def chamfer_evaluate(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size,
                     threshold, icp_correction = False):

    preds_list, pcd_list = [], []
    error_count = 0

    # get predictions and pcds
    for i in tqdm(range(len(predictions_list))):
    #for i in tqdm(range(50)):
        pcd_id = id_list[i]
        pcd, preds = cloud_list[i].transpose(1, 0), copy.deepcopy(predictions_list[i])
        #print(preds, inputs_list[i])

        preds = scale_preds(preds.tolist(), cat)
        #pcd, preds = prepare_visualisation(pcd_id, cat, i, cloud_list, predictions_list, path, ext)

        try:
            if  icp_correction:
                # note: preds are updated in place during ICP
                _, _ = icp_finetuning(o3d.utility.Vector3dVector(pcd), pcd_id, cat, preds, blueprint, temp_dir, target_dir, 
                                     ifcConvert_executable, cloudCompare_executable, sample_size, threshold, False)

            preds_list.append(preds)
            pcd_list.append(pcd)

        except Exception as e:
            print("ICP error", pcd_id, e)
            error_count += 1

    # calculate chamfer distances
    cuda = torch.device('cuda')    
    rescaled_preds = [scale_preds(preds, cat, up=0) for preds in preds_list]
    preds_t = torch.tensor(rescaled_preds, requires_grad=True, device=cuda)
    cloud_t = torch.tensor(cloud_list, device=cuda)
    
    chamfer_dists = get_chamfer_loss_tensor(preds_t, cloud_t, cat, reduce=False)
    chamfer_dists = chamfer_dists.detach().cpu().numpy()
    
#     for i, preds in enumerate(tqdm(preds_list)):
#         preds = scale_preds(preds, cat, up=0)
#         chamfer_distance, _ = get_chamfer_dist_single(pcd_list[i], preds, cat)
#         chamfer_dists.append(chamfer_distance)
        

    
    print("error_count", error_count)
    return chamfer_dists

In [ ]:
dists = chamfer_evaluate(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)
# if inference:
#     with open(model_path + 'preds_' + cat + '.pkl', 'wb') as f:
#         pickle.dump([predictions_list, id_list, dists], f)
    
plot_error_graph(dists, "Binned chamfer loss", max_val=300)

In [ ]:
# scaling up and down is required for icp calculations
def _visualise_predictions(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size,
                     threshold, icp_correction = False):

    preds_list, pcd_list = [], []
    viewer_list, ifc_list = [], []
    error_count = 0

    # get predictions and pcds
    #for i in tqdm(range(len(predictions_list))):
    for i in tqdm(range(50)):
        pcd_id = id_list[i].item()
        pcd, preds = cloud_list[i].transpose(1, 0).tolist(), copy.deepcopy(predictions_list[i])
        #print(preds, inputs_list[i])

        preds = scale_preds(preds.tolist(), cat)
        print(preds)
        #pcd, preds = prepare_visualisation(pcd_id, cat, i, cloud_list, inputs_list, ext)

#         try:
        if  icp_correction:
            # note: preds are updated in place during ICP
            viewer, ifc = icp_finetuning(o3d.utility.Vector3dVector(pcd), pcd_id, cat, preds, blueprint, temp_dir, target_dir, 
                                 ifcConvert_executable, cloudCompare_executable, sample_size, threshold, True)
        else:
            #print(preds)
            viewer, ifc = visualize_predictions([pcd], cat, [preds], blueprint, visualize=True)


        preds_list.append(preds)
        pcd_list.append(pcd)
        viewer_list.append(viewer)
        ifc_list.append(ifc)

#         except Exception as e:
#             print("ICP error", pcd_id, e)
#             error_count += 1


    print("error_count", error_count)    
    return viewer_list

In [ ]:
viewers = _visualise_predictions(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)

In [ ]:
for v in viewers:
    print(v)

In [ ]:
plot_parameter_errors(inputs_list, predictions_list, cat)

#### BP data Visualisation

In [ ]:
batch_visualise(model_path, blueprint, path, ext, device, ifc=False)

In [ ]:
merge_clouds(path, 'pipe')

### Misc

In [ ]:

# multi element Adam with single loss
def chamfer_fine_tune(n_iter, step_size, preds, cloud, cat, blueprint, alpha=1.0, visualise=True, elbow_fix=True):
    # prepare data on gpu and setup optimiser
    cuda = torch.device('cuda')
    preds_copy = copy.deepcopy(preds)
    scaled_original_preds = [scale_preds(pc.tolist(), cat) for pc in preds_copy]
    
    # temporarily change to socket category for chamfer loss calculations
    if not elbow_fix:
        cat = "socket"
        l = np.array([[p[0]*0.9] for p in preds])
        preds = list(np.hstack((preds, l)).astype(np.float32))

    preds_t = torch.tensor(preds, requires_grad=True, device=cuda)
    cloud_t = torch.tensor(cloud, device=cuda)
    optimiser = torch.optim.Adam([preds_t], lr=step_size)

    if not elbow_fix:
        a_s_t, a_c_t =  torch.clone(preds_t[:,6]), torch.clone(preds_t[:,7])

    # check initial loss
    chamfer_loss, gen_cloud = get_chamfer_loss_tensor(preds_t, cloud_t, cat, reduce=False, return_cloud=True)
    gen_cloud = gen_cloud.detach().cpu().numpy()
    #print("intial loss", chamfer_loss)   

    # iterative refinement with adam
    for i in tqdm(range (n_iter)):
        optimiser.zero_grad()
        chamfer_loss = get_chamfer_loss_tensor(preds_t, cloud_t, cat, alpha=alpha)
        chamfer_loss.backward()
        optimiser.step()
        if not elbow_fix:
            with torch.no_grad():
                preds_t[:,6], preds_t[:,7] = a_s_t, a_c_t
        
        print(i, "loss", chamfer_loss.detach().cpu().numpy())#, "preds", preds_t)
        
    # check final loss
    chamfer_loss, gen_cloud_mod = get_chamfer_loss_tensor(preds_t, cloud_t, cat, reduce=False, return_cloud=True)
    gen_cloud_mod = gen_cloud_mod.detach().cpu().numpy()          

    #print("final loss", chamfer_loss)
    modified_preds = preds_t.detach().cpu().numpy()
    
    if cat == "socket":
        cat = "elbow"
        modified_preds = modified_preds[:, :-1]
    
    # additional fix for elbows
    if elbow_fix and cat == "elbow":
        modified_preds, gen_cloud_mod = elbow_correction(n_iter, step_size*2, modified_preds, cloud, blueprint, gen_cloud_mod)
        print("error cloud", gen_cloud_mod[0].shape)
    # visualise
    if visualise:
        error_count = 0
        scaled_preds = [scale_preds(p.tolist(), cat) for p in modified_preds]
        visualisers = []
        cloud_visualisers = []

        for i, p in enumerate(scaled_preds):
            try:
                v_orignal, _ = visualize_predictions([cloud[i].transpose(1,0).tolist()], cat, [scaled_original_preds[i]], 
                                                     blueprint, visualize=True)
                v_modified, _ = visualize_predictions([None, None, cloud[i].transpose(1,0).tolist()], cat, [scaled_preds[i]], 
                                                      blueprint, visualize=True)
                #visualisers.append(v_orignal)
                visualisers.append(v_modified)
            except:
                error_count += 1
                v_orignal, _ = visualize_predictions([cloud[i].transpose(1,0).tolist(), gen_cloud[i].tolist()], cat, [], 
                                                     blueprint, visualize=True)
                v_modified, _ = visualize_predictions([None, gen_cloud_mod[i].tolist(), cloud[i].transpose(1,0).tolist()], 
                                                      cat, [], blueprint, visualize=True)
                cloud_visualisers.append(v_orignal)
                cloud_visualisers.append(v_modified)
    
#         return v_orignal,v_modified, modified_preds
        print("errors ", error_count)
        return cloud_visualisers, visualisers, modified_preds
    else:
        if elbow_fix:
            return modified_preds
        else:
            return modified_preds, chamfer_loss, gen_cloud_mod

In [ ]:
#limit = 100
limit = len(predictions_list)
torch.autograd.set_detect_anomaly(False)

if cat == 'bend':
    cat = 'elbow'
modified_preds = chamfer_fine_tune(100, 0.01, predictions_list[:limit], cloud_list[:limit], cat, blueprint, alpha=3, visualise=False)
#cl_v, v, modified_preds = chamfer_fine_tune(100, 0.01, predictions_list[:limit], cloud_list[:limit], cat, blueprint, alpha=3, visualise=True)

       

In [ ]:
print(len(cl_v), len(v))

In [ ]:
print(cl_v)

In [ ]:
#print(modified_preds[0], len(id_list))

In [ ]:
plot_parameter_errors(inputs_list, modified_preds, cat)

In [ ]:
dists = chamfer_evaluate(modified_preds, cloud_list[:limit], id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)

if inference:
    with open(model_path/('preds_finetuned_' + 'elbow' + '.pkl'), 'wb') as f:
        pickle.dump([modified_preds, id_list, dists], f)

plot_error_graph(dists, "Binned chamfer loss", max_val=300)

In [ ]:
node_dict = get_features_from_params(path)

In [ ]:
preds = torch.tensor([predictions_list[1]]).cuda()
print(predictions_list[1])
pcd = generate_tee_cloud_tensor(preds)
#tee = generate_tee_cloud(predictions_list[0])
tee = pcd[0].cpu().numpy()
tee = o3d.utility.Vector3dVector(tee)
tee_cloud = o3d.geometry.PointCloud()
tee_cloud.points = tee
o3d.io.write_point_cloud("tee_cl.pcd", tee_cloud)


In [ ]:
original = cloud_list[2]
points= o3d.utility.Vector3dVector(original.transpose(1,0))
tee_cloud.points = points
o3d.io.write_point_cloud("tee_cl_inp.pcd", tee_cloud)

In [ ]:
# undo normalisation opf the bp tee dataset created for inference, only for comparison with the inferred tee results
tee_path = 'tee_fix/tee/test/'
metadata_file = open("tee_fix/tee/metadata.json", 'r')
metadata = json.load(metadata_file)
output_path = 'tee_fix/tee/unnormalised/'

files = os.listdir(tee_path)
new_points = []
for f in tqdm(files):
    cloud_data = metadata[f.split(".")[0]]
    points = np.array(o3d.io.read_point_cloud(tee_path + f).points)
    print("a", points[0])
    print(cloud_data["norm_factor"], cloud_data["mean"])
    points *= cloud_data["norm_factor"]
    print("b", points[0])

    for i, pnt in enumerate(points):
        pnt += cloud_data["mean"]
    print("c", points[10])
    new_points.append(points)
        
new_points = o3d.utility.Vector3dVector(np.concatenate(new_points))
new_cloud = o3d.geometry.PointCloud()
new_cloud.points = new_points
o3d.io.write_point_cloud(output_path+"tee_bp_unnormalised.pcd", new_cloud)

## Mesh deformation

In [ ]:
pr = [0.5,1.0, 0.2, 10., 10., 10., math.sin(0.75), math.cos(0.75), math.sin(0.75), math.cos(0.75), math.sin(0.75), math.cos(0.75)]

pr_tensor = torch.tensor([pr]).cuda()
pcd = generate_flange_cloud_tensor(pr_tensor)
pcd = pcd[0].cpu().numpy()

#pcd = generate_flange_cloud(pr)

print(len(pcd))
new_points = o3d.utility.Vector3dVector(pcd)
new_cloud = o3d.geometry.PointCloud()
new_cloud.points = new_points
o3d.io.write_point_cloud("sample_flange.pcd", new_cloud)

